# Google Memorystore for Redis

> [Google Memorystore for Redis](https://cloud.google.com/memorystore/docs/redis/memorystore-for-redis-overview) 是一项全托管服务，由 Redis 内存数据存储提供支持，可用于构建提供亚毫秒级数据访问的应用缓存。利用 Memorystore for Redis 的 Langchain 集成，可以扩展您的数据库应用以构建由 AI 驱动的体验。

本笔记本将介绍如何使用 `MemorystoreVectorStore` 类将向量嵌入存储到 [Memorystore for Redis](https://cloud.google.com/memorystore/docs/redis/memorystore-for-redis-overview) 中。

在 [GitHub](https://github.com/googleapis/langchain-google-memorystore-redis-python/) 上了解有关该软件包的更多信息。

[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-memorystore-redis-python/blob/main/docs/vector_store.ipynb)

## 先决条件

## 开始之前

要运行此笔记本，您需要执行以下操作：

*   [创建 Google Cloud 项目](https://developers.google.com/workspace/guides/create-project)
*   [启用 Memorystore for Redis API](https://console.cloud.google.com/flows/enableapi?apiid=redis.googleapis.com)
*   [创建 Memorystore for Redis 实例](https://cloud.google.com/memorystore/docs/redis/create-instance-console)。请确保版本大于或等于 7.2。

### 🦜🔗 库安装

该集成存在于自己的 `langchain-google-memorystore-redis` 包中，因此我们需要安装它。

In [ ]:
%pip install -upgrade --quiet langchain-google-memorystore-redis langchain

**仅限 Colab：** 取消注释下面的单元格以重启内核，或使用按钮重启内核。对于 Vertex AI Workbench，您可以使用顶部的按钮重启终端。

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### ☁ 设置您的 Google Cloud 项目
设置您的 Google Cloud 项目，以便在此 notebook 中利用 Google Cloud 资源。

如果您不知道项目 ID，请尝试以下方法：

* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 请参阅支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### 🔐 认证
使用此笔记本中登录的 IAM 用户对 Google Cloud 进行身份验证，以便访问您的 Google Cloud 项目。

* 如果你正在使用 Colab 运行此笔记本，请使用下面的单元格并继续。
* 如果你正在使用 Vertex AI Workbench，请查看此处的设置说明：[https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)。

In [ ]:
from google.colab import auth

auth.authenticate_user()

## 基本用法

### 初始化向量索引

In [13]:
import redis
from langchain_google_memorystore_redis import (
    DistanceStrategy,
    HNSWConfig,
    RedisVectorStore,
)

# Connect to a Memorystore for Redis instance
redis_client = redis.from_url("redis://127.0.0.1:6379")

# Configure HNSW index with descriptive parameters
index_config = HNSWConfig(
    name="my_vector_index", distance_strategy=DistanceStrategy.COSINE, vector_size=128
)

# Initialize/create the vector store index
RedisVectorStore.init_index(client=redis_client, index_config=index_config)

### 准备文档

文本在与向量存储交互之前需要进行处理和数值表示。这包括：

* 加载文本：TextLoader 从文件（例如“state_of_the_union.txt”）获取文本数据。
* 文本拆分：CharacterTextSplitter 将文本分解成更小的块，以适应嵌入模型。

In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("./state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

### 将文档添加到向量数据库

在文本准备和嵌入生成之后，以下方法会将它们插入到 Redis 向量数据库中。

#### 方法 1：用于直接插入的类方法

此方法使用 `from_documents` 类方法将嵌入创建和插入合并为单个步骤：

In [ ]:
from langchain_community.embeddings.fake import FakeEmbeddings

embeddings = FakeEmbeddings(size=128)
redis_client = redis.from_url("redis://127.0.0.1:6379")
rvs = RedisVectorStore.from_documents(
    docs, embedding=embeddings, client=redis_client, index_name="my_vector_index"
)

#### 方法二：基于实例的插入
这种方法在处理新的或现有的 RedisVectorStore 时提供了灵活性：

* [可选] 创建一个 RedisVectorStore 实例：实例化一个 RedisVectorStore 对象以进行自定义。如果您已经有一个实例，请继续下一步。
* 添加带元数据的文本：向实例提供原始文本和元数据。嵌入生成和插入到向量存储中的过程会自动处理。

In [ ]:
rvs = RedisVectorStore(
    client=redis_client, index_name="my_vector_index", embeddings=embeddings
)
ids = rvs.add_texts(
    texts=[d.page_content for d in docs], metadatas=[d.metadata for d in docs]
)

### 执行相似性搜索 (KNN)

在填充了向量存储后，就可以搜索与查询在语义上相似的文本了。以下是如何使用带有默认设置的 KNN（K 最近邻）：

* 构思查询：用自然语言提问来表达搜索意图（例如，“总统对 Ketanji Brown Jackson 发表了什么看法”）。
* 检索相似结果：`similarity_search` 方法会查找向量存储中含义上最接近查询的项目。

In [ ]:
import pprint

query = "What did the president say about Ketanji Brown Jackson"
knn_results = rvs.similarity_search(query=query)
pprint.pprint(knn_results)

### 执行基于范围的相似性搜索

范围查询通过指定期望的相似性阈值和查询文本来提供更多控制：

*   制定查询：自然语言问题定义搜索意图。
*   设置相似性阈值：`distance_threshold` 参数决定了匹配项在多大程度上被视为相关。
*   检索结果：`similarity_search_with_score` 方法从向量存储中查找位于指定相似性阈值内的项。

In [ ]:
rq_results = rvs.similarity_search_with_score(query=query, distance_threshold=0.8)
pprint.pprint(rq_results)

### 执行最大边际相关性 (MMR) 搜索

MMR 查询旨在查找与查询相关且彼此多样化的结果，从而减少搜索结果中的冗余。

*   **构建查询**: 自然语言问题定义搜索意图。
*   **平衡相关性和多样性**: `lambda_mult` 参数控制严格相关性与促进结果多样性之间的权衡。
*   **检索 MMR 结果**: `max_marginal_relevance_search` 方法根据 lambda 设置返回优化相关性和多样性组合的条目。

In [ ]:
mmr_results = rvs.max_marginal_relevance_search(query=query, lambda_mult=0.90)
pprint.pprint(mmr_results)

## 将向量存储用作检索器

为了与 LangChain 的其他组件无缝集成，可以将向量存储转换为检索器。这具有多种优势：

* LangChain 兼容性：许多 LangChain 工具和方法都设计为直接与检索器交互。
* 使用方便：`as_retriever()` 方法将向量存储转换为一种格式，简化了查询。

In [ ]:
retriever = rvs.as_retriever()
results = retriever.invoke(query)
pprint.pprint(results)

## 清理

### 从向量存储中删除文档

有时，有必要从向量存储中删除文档（及其关联的向量）。`delete` 方法提供了此功能。

In [ ]:
rvs.delete(ids)

### 删除向量索引

在某些情况下，需要删除现有的向量索引。常见原因包括：

* 索引配置更改：如果需要修改索引参数，通常需要删除并重新创建索引。
* 存储管理：删除未使用的索引有助于释放 Redis 实例中的空间。

注意：删除向量索引是一项不可逆的操作。在继续之前，请确保不再需要存储的向量和搜索功能。

In [22]:
# Delete the vector index
RedisVectorStore.drop_index(client=redis_client, index_name="my_vector_index")